In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
import matplotlib
import seaborn as sns
import scipy
from scipy.stats import normaltest, skew, kurtosis

In [ ]:
df = pd.read_csv('ScorePerDay20', index_col='dates')

In [ ]:
#creating month and week labels for EDA

df['dates'] = pd.to_datetime(df.index)
df['dates'] = pd.to_datetime(df['dates'])
df['Month'] = df['dates'].dt.month

In [ ]:
#aggregating scores per tweet into scores per day

SDay = {}
SDates = {}
ScoreDay = []
ScorePerDay = {}

for index, dates in enumerate(df.index):
    if dates not in SDay:
        SDay[dates] = 0
        SDates[dates] = 0
    if dates in SDay:
        SDay[dates] += df['score'][index]
        SDates[dates] +=1
        

for items in zip(SDay.values(), SDates.values()):
    ScoreDay.append(items[0]/items[1])

for items in zip(SDates.keys(), ScoreDay):
    ScorePerDay[items[0]] = items[1]


In [ ]:
#postive/neutral/negative scores per day

positive = 0
negative = 0
neutral = 0
for tweet in df['score']:
    if tweet > 0:
        positive +=1 
    if tweet < 0:
        negative +=1
    if tweet == 0:
        neutral +=1
    
print(positive, negative, neutral)
PNN = pd.DataFrame(data=[positive, negative, neutral], index=['pos','neg','neu'])
print(positive/total, neutral/total, negative/total)

In [ ]:
#normality and skewness test 

print(skew(df['score']))
print(kurtosis(df['score']))
print(normaltest(df['score']))

In [ ]:
#outlier detection

mean = np.mean(df3)
std = np.std(df3)
print(mean+(std*3))

troughs = []
for index, i in enumerate( df['score']):
    if i < -0.12:
        print(df.iloc[index],index)
    if i>0.14:
        
        print(df.iloc[index])

In [ ]:
#calculating mean scores per day of the week

MeanDays = df.groupby(['Weekday']).mean()

In [ ]:
#plotting score per day of the week


keys = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
scores = [-0.070756,-0.070523,-0.067389,-0.059437,-0.045015, -0.042870,-0.046607]

palette = sns.color_palette('BrBG',6) + sns.color_palette('BrBG',4)
sns.palplot(palette)
a = sns.barplot(keys,scores,palette = palette)
sns.lineplot(keys,scores)
plt.xticks(rotation=45)
a.invert_yaxis()
plt.ylabel('Negative Score')
sns.set(rc={"figure.dpi":300})

In [ ]:
#creating descriptors 

data = [i for i in ScorePerDay.values()]
std = np.std(data)
mean =  np.mean(data)
min_ = min(data)
max_ = max(data)
index = ['std','mean','min','max']
descriptors = pd.DataFrame((std,mean,min_,max_), columns=['Value'], index=index)

In [ ]:
#creating score per day distribution plot

sns.displot(df,kde=True, legend=False)
plt.ylabel('Number of Days')
plt.xlabel('Polarity Score')
sns.set(rc={"figure.dpi":600})

In [ ]:
#calculating the pandemic related terms by day

Pwords = {}
covid_words = ['pandemia', 'mascarillas', 'virus', 'cuarentena', 'covid', 'casos','gobierno','crisis','muertos','salud','mundo','personas','hospitales','crisis','vida','contagio','fallecidos' ]

for index, words in zip(df.index, df['CleanText']):
    
    if index not in Pwords:
        Pwords[index] = 0
    if index in Pwords:
        try:
            for w in words.split():
                w= str(w)
                w = w.lower()
                if w in covid_words:
                    Pwords[index] += 1
        except:
            print(w)

In [ ]:
#normalizing pandemic terms by tweets per day

PropPword = {}


for words,dates in zip(Pwords.items(),SDates.items()):
    PropPword[words[0]] = words[1]/dates[1]

In [ ]:
#calculating the number of positve,negative,neutral tweets per day

Mneg = {}
Mpos = {}
Mneu = {}

for index, dates in enumerate(df['new_dates']):
    if dates not in Mneg:
        Mneg[dates] = 0
    if dates in Mneg:
        if df['score'][index] <0:
            Mneg[dates] +=1
        
for index, dates in enumerate(df['new_dates']):
    if dates not in Mpos:
        Mpos[dates] = 0
    if dates in Mneg:
        if df['score'][index] >0:
            Mpos[dates] +=1
            
for index, dates in enumerate(df['new_dates']):
    if dates not in Mneu:
        Mneu[dates] = 0
    if dates in Mneu:
        if df['score'][index] ==0:
            Mneu[dates] +=1

In [ ]:
#averaging the results

MnegNor = {}
MposNor = {}
MneuNor = {}


for dates,tags in zip(SDates.items(),Mneg.items()):
    MnegNor[dates[0]] = tags[1]/dates[1]

for dates,tags in zip(SDates.items(),Mneu.items()):
    MneuNor[dates[0]] = tags[1]/dates[1]
    
for dates,tags in zip(SDates.items(),Mpos.items()):
    MposNor[dates[0]] = tags[1]/dates[1]

In [ ]:
#proportion of postive/negative tweets per day

PropPosNeg = {}

for pos,neg in zip(MposNor.items(),MnegNor.items()):
    PropPosNeg[pos[0]] = pos[1]/neg[1]

In [ ]:
#saving all propotions in csv files

df2= pd.DataFrame.from_dict(PropPosNeg.values())
df2['dates'] = MnegNor.keys()
df2 = df2.set_index('dates')
df2.to_csv('PropPosNeg', index='dates')

In [ ]:
#visualizing proportions 

figure(figsize=(16, 12), dpi=80)
plt.xticks(rotation=90, fontsize=18)
plt.scatter(MnegNor.keys(), MnegNor.values())
plt.plot(MnegNor.keys(), MnegNor.values(), label='neg')
plt.scatter(MposNor.keys(), MposNor.values())
plt.plot(MposNor.keys(), MposNor.values(), label='pos')
plt.scatter(MneuNor.keys(), MneuNor.values())
plt.plot(MneuNor.keys(), MneuNor.values(), label='neu')
plt.grid()
plt.legend()

In [ ]:
#final visualizations of erros and outliers showned in the Master thesis

index = pd.date_range(start = "2020-01-01", end = "2021-12-31", freq = "D")
index = [pd.to_datetime(date, format='%Y-%m-%d').date() for date in index]
polarity = da
df2 = pd.DataFrame(data=polarity,index=index, columns=['Polarity'])
print (df2.head())

ax = df2.plot(color='steelblue')
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%Y'))
plt.ylabel('Polarity Score')
plt.plot(index, df['score'].rolling(25).mean(),label='Rolling Mean',color='darkorange')
plt.xticks(rotation=45)
plt.legend()
plt.xlim('2019-12-15','2022-01-12')
point1 = ['2020-03-12', -0.19]
point2 = ['2020-04-26', -0.19]
point3 = ['2020-12-31', 0.17]
point4 = ['2021-12-31', 0.17]
point5 = ['2020-06-02', -0.17]
point6 = ['2021-01-19', -0.125]


ax.annotate('c1', xy=point1, xytext=(-30, 30),

   textcoords='offset points',
   color='black', size='small',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.2,head_width=0.5,head_length=0.3',
      facecolor='r')#, #shrinkB=radius * 1.2)
   )
ax.annotate('c2', xy=point2, xytext=(20, 0),

   textcoords='offset points',
   color='black', size='small',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.2,head_width=0.5,head_length=0.3',
      facecolor='r')#, #shrinkB=radius * 1.2)
   )
ax.annotate('h1', xy=point3, xytext=(-30, -30),

   textcoords='offset points',
   color='black', size='small',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.2,head_width=0.5,head_length=0.3',
      facecolor='b')#, #shrinkB=radius * 1.2)
   )
ax.annotate('h2', xy=point4, xytext=(-30, -30),

   textcoords='offset points',
   color='black', size='small',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.2,head_width=0.5,head_length=0.3',
      facecolor='b')#, #shrinkB=radius * 1.2)
   )
ax.annotate('c3', xy=point5, xytext=(20, 0),
   textcoords='offset points',
   color='black', size='small',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.2,head_width=0.5,head_length=0.3',
      facecolor='r')
   )
ax.annotate('c4', xy=point6, xytext=(20, -10),
   textcoords='offset points',
   color='black', size='small',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.2,head_width=0.5,head_length=0.3',
      facecolor='r')
   )
sns.set(rc={"figure.dpi":300})
plt.show()

In [ ]:
index = pd.date_range(start = "2020-01-01", end = "2021-12-31", freq = "D")
index = [pd.to_datetime(date, format='%Y-%m-%d').date() for date in index]
polarity = da
df2 = pd.DataFrame(data=polarity,index=index, columns=['Polarity'])
ax = df2.plot(color='steelblue')
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%Y'))
plt.ylabel('Polarity Score')
plt.plot(index, df['score'].rolling(25).mean(),label='Rolling Mean',color='darkorange')
plt.xticks(rotation=45)
plt.legend()
plt.xlim('2020-01-01','2021-12-31')
point1 = ['2020-03-12', -0.19]
point2 = ['2020-04-26', -0.19]
point3 = ['2020-12-31', 0.16]
point3 = ['2020-12-31', 0.16]


ax.annotate('03/12', xy=point1, xytext=(-35, 30),

   textcoords='offset points',
   color='green', size='large',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.1,head_width=0.3,head_length=0.3',
      facecolor='r')
   )
ax.annotate('Circled Marker', xy=point2, xytext=(-60, -20),

   textcoords='offset points',
   color='green', size='large',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.1,head_width=0.3,head_length=0.3',
      facecolor='r')
   )
ax.annotate('Circled Marker', xy=point3, xytext=(-60, -30),

   textcoords='offset points',
   color='green', size='large',
   arrowprops=dict(
      arrowstyle='simple,tail_width=0.1,head_width=0.3,head_length=0.3',
      facecolor='r')
   )
plt.show()